In [139]:
import matplotlib.pyplot as plt
import math

In [2]:
def unzip(l):
    return [x[0] for x in l], [x[1] for x in l]

In [43]:
class SolutionMethod:
    def __init__(self, left_x, initial_y, right_x, step, func):
        self.left_x = left_x
        self.right_x = right_x
        self.initial_y = initial_y
        self.step = step
        self.func = func
        
        self.steps = int((right_x - left_x) // step + 1)
        
    def produce_next_y(self, x, y):
        raise NotImplementedError
        
    def produce_xy_pairs(self):
        x, y = self.left_x, self.initial_y
        for _ in range(self.steps):
            yield x, y
            y = self.produce_next_y(x, y)
            x += self.step
            
            
class EulerMethod(SolutionMethod):
    name = 'Euler method'
    
    def produce_next_y(self, x, y):
        return y + self.step * self.func(x, y)
    

class ImprovedEulerMethod(SolutionMethod):
    name = 'Improved Euler method'
    
    def produce_next_y(self, x, y):
        m1 = self.func(x, y)
        m2 = self.func(x + self.step, y + self.step * m1)
        return y + self.step * (m1 + m2) / 2
    

class RungeKuttaMethod(SolutionMethod):
    name = 'Runge-Kutta method'
    
    def produce_next_y(self, x, y):
        k1 = self.step * self.func(x, y)
        k2 = self.step * self.func(x + self.step / 2, y + k1 / 2)
        k3 = self.step * self.func(x + self.step / 2, y + k2 / 2)
        k4 = self.step * self.func(x + self.step, y + k3)
        return y + (1 / 6) * (k1 + 2 * k2 + 2 * k3 + k4)
    
    
class Exact(SolutionMethod):
    name = 'Exact'
    
    def exact_c(self):
        return (
            self.initial_y - (
                3 * self.left_x * math.exp(self.left_x) / 2
            )
        ) / (self.left_x * math.exp(-self.left_x))
    
    def produce_y(self, x, y):
        return self.ivp_c * math.exp(-x) * x + 3 * x * math.exp(x) / 2
    
    def produce_xy_pairs(self):
        self.ivp_c = self.exact_c()
        x, y = self.left_x, self.initial_y
        for _ in range(self.steps):
            yield x, y
            x += self.step
            y = self.produce_y(x, y)
    

In [127]:
class MethodData:
    def __init__(self, instance, eager_exact_data=None, eager_global_n_begin=None, eager_global_n_end=None,
                 eager_global_n_step=None):
        self.instance = instance
        self.name = self.instance.name
        self.data_x = []
        self.data_y = []
        
        self.exact_data = eager_exact_data
        
        self.calculate_data()
        if eager_exact_data:
            self.calculate_local_error(eager_exact_data)
            if eager_global_n_begin and eager_global_n_end:
                self.calculate_global_error(eager_exact_data, eager_global_n_begin, eager_global_n_end,
                                            eager_global_n_step)
    
    def calculate_data(self, save=True):
        data_x, data_y = unzip(list(self.instance.produce_xy_pairs()))
        if save:
            self.data_x = data_x
            self.data_y = data_y
        return data_x, data_y
    
    def calculate_local_error(self, exact_data_y, for_data=None, save=True):
        if for_data is None:
            for_data = self.data_y
        local_error_data = [abs(data_point - exact_point) for data_point, exact_point in zip(for_data, exact_data_y)]
        if save:
            self.local_error_data = local_error_data
        return local_error_data
    
    def calculate_global_error(self, exact_data_y, n_begin, n_end, n_step):
        global_error_x = []
        global_error_y = []
        for n in range(n_begin, n_end, n_step):
            step = float(self.instance.right_x - self.instance.left_x) / n
            self.instance.__init__(
                self.instance.left_x,
                self.instance.initial_y,
                self.instance.right_x,
                step,
                self.instance.func
            )
            exact_instance = Exact(
                self.instance.left_x,
                self.instance.initial_y,
                self.instance.right_x,
                step,
                self.instance.func
            )
            exact_data = MethodData(exact_instance)
            data_x, data_y = self.calculate_data(save=False)
            local_error_y = self.calculate_local_error(exact_data.data_y, for_data=data_y, save=False)
            global_error_x.append(n)
            global_error_y.append(local_error_y[-1])
        self.global_error_x = global_error_x
        self.global_error_y = global_error_y
        return global_error_x, global_error_y

    
    def display_data_on(self, subplot):
        subplot.plot(self.data_x, self.data_y, label=self.name)
    
    def display_local_error_on(self, subplot):
        subplot.plot(self.data_x, self.local_error_data, label=self.name)
        
    def display_global_error_on(self, subplot):
        subplot.plot(self.global_error_x, self.global_error_y, label=self.name)


In [136]:
def display_method_data(figure, idx, title, data_list, invoker):
    subplot = plt.plot(3, 1, idx)
    subplot.set_title(title)
    for method in data_list:
        invoker(subplot, method)

def run(left_x, initial_y, right_x, step, func, n_begin, n_end, n_step):
    euler_instance = EulerMethod(left_x, initial_y, right_x, step, func)
    imp_euler_instance = ImprovedEulerMethod(left_x, initial_y, right_x, step, func)
    runge_kutta_instance = RungeKuttaMethod(left_x, initial_y, right_x, step, func)
    exact_instance = Exact(left_x, initial_y, right_x, step, func)
        
    exact_data = MethodData(exact_instance)
    euler_data = MethodData(euler_instance, exact_data.data_y, n_begin, n_end, n_step)
    imp_euler_data = MethodData(imp_euler_instance, exact_data.data_y, n_begin, n_end, n_step)
    runge_kutta_data = MethodData(runge_kutta_instance, exact_data.data_y, n_begin, n_end, n_step)
    
    figure = plt.figure(figsize=(14, 30))
    display_method_data(figure, 1, 'Solution for y(x)', [
        euler_data, 
        imp_euler_data, 
        runge_kutta_data, 
        exact_data
    ], lambda x, y: y.display_data_on(x))
    display_method_data(figure, 2, 'Local truncation error', [
        euler_data,
        imp_euler_data, 
        runge_kutta_data
    ], lambda x, y: y.display_local_error_on(x))
    display_method_data(figure, 3, 'Global truncation error', [
        euler_data,
        imp_euler_data,
        runge_kutta_data
    ], lambda x, y: y.display_global_error_on(x))

    plt.legend()
    plt.show()

In [137]:
def f(x, y):
    return 3 * x * math.exp(x) - y * (1 - 1 / x)

In [138]:
run(7, 0.1, 11, 0.1, f, 1, 20, 1)

TraitError: Dimension mismatch for trait x of class <class 'bqplot.marks.Lines'>: expected an             array of dimension comprised in interval [1, 2] and got an array of shape ()